In [1]:
pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.9/321.9 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
import joblib
import gradio as gr
import os
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
tickers = [
    'RELIANCE.NS', 'TCS.NS', 'INFY.NS', 'HDFCBANK.NS', 'ICICIBANK.NS',
    'HINDUNILVR.NS', 'ITC.NS', 'KOTAKBANK.NS', 'LT.NS', 'SBIN.NS',
    'BHARTIARTL.NS', 'ASIANPAINT.NS', 'HCLTECH.NS', 'MARUTI.NS', 'AXISBANK.NS',
    'BAJFINANCE.NS', 'WIPRO.NS', 'SUNPHARMA.NS', 'NESTLEIND.NS', 'ULTRACEMCO.NS',
    'TITAN.NS', 'TATASTEEL.NS', 'TECHM.NS', 'POWERGRID.NS', 'INDUSINDBK.NS',
    'M&M.NS', 'NTPC.NS', 'HDFCLIFE.NS', 'ONGC.NS', 'JSWSTEEL.NS',
    'BAJAJFINSV.NS', 'GRASIM.NS', 'CIPLA.NS', 'SBILIFE.NS', 'TATAMOTORS.NS',
    'COALINDIA.NS', 'ADANIPORTS.NS', 'BPCL.NS', 'DIVISLAB.NS', 'BRITANNIA.NS',
    'APOLLOHOSP.NS', 'HEROMOTOCO.NS', 'EICHERMOT.NS', 'UPL.NS', 'DRREDDY.NS',
    'IOC.NS', 'HINDALCO.NS', 'BAJAJ-AUTO.NS', 'TATACONSUM.NS', 'SHREECEM.NS'
]

In [4]:
def fetch_data(ticker):
    data = yf.download(ticker, start='2020-01-01', end='2025-01-01')
    return data


def train_model(ticker):
    print(f"Training model for {ticker}...")
    try:
        data = fetch_data(ticker)
        if data.empty:
            return f"Error: No data found for {ticker}."

        data['Next_Close'] = data['Close'].shift(-1)
        data.dropna(inplace=True)

        X = data[['Open', 'High', 'Low', 'Close', 'Volume']]
        y = data['Next_Close']

        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)

        X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

        model = LinearRegression()
        model.fit(X_train, y_train)

        joblib.dump((model, scaler), f'{ticker}_model.pkl')
        return f'Model trained and saved for {ticker}.'

    except Exception as e:
        return f"Error training model for {ticker}: {str(e)}"

In [5]:

def predict_price(ticker):
    try:
        if not os.path.exists(f'{ticker}_model.pkl'):
            print(f"Model file for {ticker} not found. Training now...")
            train_model(ticker)

        model, scaler = joblib.load(f'{ticker}_model.pkl')

        data = fetch_data(ticker)
        latest_data = data.iloc[-1][['Open', 'High', 'Low', 'Close', 'Volume']].values.reshape(1, -1)
        latest_data_scaled = scaler.transform(latest_data)
        prediction = model.predict(latest_data_scaled)

        return f'Predicted Next Close Price for {ticker}: {prediction[0]:.2f}'

    except Exception as e:
        return f"Error: {str(e)}"

In [8]:
import yfinance as yf
import joblib
import gradio as gr
import os
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pandas as pd


def fetch_data(ticker):
    try:
        data = yf.download(ticker, period='1y')
        if data.empty:
            raise ValueError("No data found for the selected ticker.")
        return data
    except Exception as e:
        return f"Error fetching data: {str(e)}"


def train_model(ticker):
    try:
        data = fetch_data(ticker)
        if isinstance(data, str):
            return data

        data['Next_Close'] = data['Close'].shift(-1)
        data.dropna(inplace=True)

        X = data[['Open', 'High', 'Low', 'Close', 'Volume']]
        y = data['Next_Close']

        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)

        X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

        model = LinearRegression()
        model.fit(X_train, y_train)

        joblib.dump((model, scaler), f'{ticker}_model.pkl')
        return f'Model trained and saved for {ticker}.'
    except Exception as e:
        return f"Error training model for {ticker}: {str(e)}"


def predict_price(ticker):
    try:
        model_file = f'{ticker}_model.pkl'
        if not os.path.exists(model_file):
            result = train_model(ticker)
            if "Error" in result:
                return result

        model, scaler = joblib.load(model_file)
        data = fetch_data(ticker)
        if isinstance(data, str):
            return data

        latest_data = data.iloc[-1][['Open', 'High', 'Low', 'Close', 'Volume']].values.reshape(1, -1)
        latest_data_scaled = scaler.transform(latest_data)
        prediction = model.predict(latest_data_scaled)

        return f'Predicted Next Close Price for {ticker}: {prediction[0]:.2f}'

    except Exception as e:
        return f"Error during prediction: {str(e)}"


def interface_predict(ticker):
    return predict_price(ticker)


ticker_list = [
    'RELIANCE.NS', 'TCS.NS', 'INFY.NS', 'HDFCBANK.NS', 'ICICIBANK.NS',
    'HINDUNILVR.NS', 'ITC.NS', 'KOTAKBANK.NS', 'LT.NS', 'SBIN.NS',
    'BHARTIARTL.NS', 'ASIANPAINT.NS', 'HCLTECH.NS', 'MARUTI.NS', 'AXISBANK.NS',
    'BAJFINANCE.NS', 'WIPRO.NS', 'SUNPHARMA.NS', 'NESTLEIND.NS', 'ULTRACEMCO.NS',
    'TITAN.NS', 'TATASTEEL.NS', 'TECHM.NS', 'POWERGRID.NS', 'INDUSINDBK.NS',
    'M&M.NS', 'NTPC.NS', 'HDFCLIFE.NS', 'ONGC.NS', 'JSWSTEEL.NS',
    'BAJAJFINSV.NS', 'GRASIM.NS', 'CIPLA.NS', 'SBILIFE.NS', 'TATAMOTORS.NS',
    'COALINDIA.NS', 'ADANIPORTS.NS', 'BPCL.NS', 'DIVISLAB.NS', 'BRITANNIA.NS',
    'APOLLOHOSP.NS', 'HEROMOTOCO.NS', 'EICHERMOT.NS', 'UPL.NS', 'DRREDDY.NS',
    'IOC.NS', 'HINDALCO.NS', 'BAJAJ-AUTO.NS', 'TATACONSUM.NS', 'SHREECEM.NS'
]


dropdown = gr.Dropdown(label="Select Company", choices=ticker_list, value="RELIANCE.NS")
output_text = gr.Textbox(label="Predicted Stock Price")




gui = gr.Interface(
    fn=interface_predict,
    inputs=dropdown,
    outputs=output_text,
    title="📈💰 Stock Price Prediction - Linear Regression 🚀📊",
    description=(
        "🚨 **Disclaimer** 🚨\n"
        "This student project is for learning purposes only. Stock predictions shown are based on basic models and are not financial advice. "
        "We are not SEBI/NSE-registered investors. Do your own research or consult a professional before investing.\n\n"
        "Select a company to predict its next closing stock price.\n\n"
    )
)





if __name__ == "__main__":
    gui.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6f0980e36ba2c39d11.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
